### Data

In [1]:
# Импортируем необходимые библиотеки
import os
from getpass import getpass
from IPython.display import clear_output
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
!pip install catboost==0.24.4
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
from catboost import CatBoostClassifier, Pool
import joblib
import torch.nn as nn
import torch
import warnings
warnings.filterwarnings("ignore")
clear_output()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
# Клонируем репозиторий с решением
!git clone https://github.com/SlayZar/alpha_task_2.git

Cloning into 'alpha_task_2'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (134/134), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 134 (delta 70), reused 43 (delta 13), pack-reused 0
Receiving objects: 100% (134/134), 7.00 MiB | 19.80 MiB/s, done.
Resolving deltas: 100% (70/70), done.


In [3]:
TEST_TRANSACTIONS_PATH = 'alpha_task_2/test_transactions_contest/'
TEST_TARGET_PATH = 'alpha_task_2/alfabattle2_test_target_contest.csv'
MODEL_PATH = 'alpha_task_2/models'
SOL_PATH = 'alpha_task_2/sols'
TEST_BACKETS_PATH = 'alpha_task_2/test_buckets'

In [4]:
%%time
# Скачиваем исходные тестовые данные и файлы моделей

! rm -r alpha_task_2/sols # Папка, в которую будут записываться файлы с решениями
! mkdir alpha_task_2/sols
! rm -r alpha_task_2/test_buckets 
! mkdir alpha_task_2/test_buckets

links2 = ["https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/KxlW45P4XfUt-Q",
          "https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/ueK2vNL95FWwCQ",
          "https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/cMj0ajcwD849Qg",
          "https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/wrmgvK1GYT3LQw",
          "https://getfile.dokpub.com/yandex/get/https://yadi.sk/d/QZ7Xydi4u2ILgw"] 
          
os.system("wget -O alpha_task_2/test_df.zip {}".format(links2[0]))
os.system(f"wget -O {MODEL_PATH}/model_9 {links2[1]}")
os.system(f"wget -O {MODEL_PATH}/model_10 {links2[2]}")
os.system(f"wget -O {MODEL_PATH}/model_12 {links2[3]}")
os.system(f"wget -O {MODEL_PATH}/model_13 {links2[4]}")

!unzip alpha_task_2/test_df.zip -d alpha_task_2/
!rm alpha_task_2/test_df.zip

clear_output()

CPU times: user 162 ms, sys: 108 ms, total: 270 ms
Wall time: 9min 13s


# Scoring

In [6]:
from alpha_task_2.preparation_funcs import __amnt_pivot_table_by_column_as_frame, preprocess, \
  create_buckets_from_transactions, nn_scoring, boost_scor, agg_scoring

In [7]:
TEST_TRANSACTIONS_PATH = 'alpha_task_2/test_transactions_contest/'
TEST_TARGET_PATH = 'alpha_task_2/alfabattle2_test_target_contest.csv'
SOL_PATH = 'alpha_task_2/sols'
TEST_BACKETS_PATH = 'alpha_task_2/test_buckets'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [8]:
test_targ = pd.read_csv(TEST_TARGET_PATH)

In [10]:
def scoring():
  print('Boosting preparation')
  #####################################
  # Boosting preparation
  #####################################
  test_scores = pd.DataFrame()
  for i in tqdm(os.listdir(f'{TEST_TRANSACTIONS_PATH}')):
    tmp_df = pd.read_parquet(f'{TEST_TRANSACTIONS_PATH}{i}')
    tmp_df = preprocess(tmp_df)
    tmp_df = tmp_df.merge(test_targ, on=['app_id'], how='left')
    test_scores = pd.concat([test_scores, tmp_df])

  print('Boosting scores')
  #####################################
  # Boosting scores
  #####################################
  for i in [9, 10, 12, 13]:
    boost_scor(test_scores, test_targ, MODEL_PATH, SOL_PATH, i)

  print('NN preparation')
  #####################################
  # NN preparation
  #####################################

  create_buckets_from_transactions(TEST_TRANSACTIONS_PATH, 
                                  save_to_path=TEST_BACKETS_PATH, frame_with_ids=test_targ, 
                                  num_parts_to_preprocess_at_once=5, num_parts_total=50, has_target=False)

  print('NN scores')
  #####################################
  # NN scores
  #####################################

  for i in range(3):
    nn_scoring(TEST_BACKETS_PATH, MODEL_PATH, f'nn_model_{i+1}.pt',
              f'alpha_task_2/sols/nn_mod{i+1}.csv', device)

  print('Aggregation models')
  #####################################
  # Aggregation models
  #####################################

  sols = agg_scoring(SOL_PATH, MODEL_PATH)
  print(f'Submit file saved in {SOL_PATH}/final_solution.csv')
  return sols

In [11]:
%%time
res = scoring()

Boosting preparation



Boosting scores
NN preparation


Reading chunks:

alpha_task_2/test_transactions_contest/part_000_1063620_to_1074462.parquet
alpha_task_2/test_transactions_contest/part_001_1074463_to_1085303.parquet
alpha_task_2/test_transactions_contest/part_002_1085304_to_1095174.parquet
alpha_task_2/test_transactions_contest/part_003_1095175_to_1105002.parquet
alpha_task_2/test_transactions_contest/part_004_1105003_to_1116054.parquet



Reading chunks:

alpha_task_2/test_transactions_contest/part_005_1116055_to_1127527.parquet
alpha_task_2/test_transactions_contest/part_006_1127528_to_1137672.parquet
alpha_task_2/test_transactions_contest/part_007_1137673_to_1147504.parquet
alpha_task_2/test_transactions_contest/part_008_1147505_to_1157749.parquet
alpha_task_2/test_transactions_contest/part_009_1157750_to_1167980.parquet



Reading chunks:

alpha_task_2/test_transactions_contest/part_010_1167981_to_1178851.parquet
alpha_task_2/test_transactions_contest/part_011_1178852_to_1190630.parquet
alpha_task_2/test_transactions_contest/part_012_1190631_to_1200939.parquet
alpha_task_2/test_transactions_contest/part_013_1200940_to_1211425.parquet
alpha_task_2/test_transactions_contest/part_014_1211426_to_1222122.parquet


...


Reading chunks:

alpha_task_2/test_transactions_contest/part_035_1426764_to_1436400.parquet
alpha_task_2/test_transactions_contest/part_036_1436401_to_1448080.parquet
alpha_task_2/test_transactions_contest/part_037_1448081_to_1459730.parquet
alpha_task_2/test_transactions_contest/part_038_1459731_to_1470134.parquet
alpha_task_2/test_transactions_contest/part_039_1470135_to_1479802.parquet



Reading chunks:

alpha_task_2/test_transactions_contest/part_040_1479803_to_1489232.parquet
alpha_task_2/test_transactions_contest/part_041_1489233_to_1499712.parquet
alpha_task_2/test_transactions_contest/part_042_1499713_to_1510447.parquet
alpha_task_2/test_transactions_contest/part_043_1510448_to_1520793.parquet
alpha_task_2/test_transactions_contest/part_044_1520794_to_1531282.parquet



Reading chunks:

alpha_task_2/test_transactions_contest/part_045_1531283_to_1541445.parquet
alpha_task_2/test_transactions_contest/part_046_1541446_to_1551040.parquet
alpha_task_2/test_transactions_contest/part_047_1551041_to_1560328.parquet
alpha_task_2/test_transactions_contest/part_048_1560329_to_1570341.parquet
alpha_task_2/test_transactions_contest/part_049_1570342_to_1580442.parquet




NN scores



Aggregation models
Submit file saved in alpha_task_2/sols/final_solution.csv
CPU times: user 1h 23min, sys: 2min 29s, total: 1h 25min 29s
Wall time: 1h 37min 47s
